In [510]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import aries_cloudagent.wallet.util as util
import base64
import base58
import uuid

agent1 = {}
agent2 = {}
plugin = {}

## Enter Invite Url:

In [511]:
agent1['url'] = "http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNDQwODVkNzktYmIxYS00OGYyLWJkZDUtZGYwYjE5MGUxMjI3IiwgImxhYmVsIjogIkNsaWVudCAoYWRtaW4pIiwgInJlY2lwaWVudEtleXMiOiBbIndhWEduNnh1ZG5ybVFiYTVqTWpxTmVuclExMnY1VDFrN1hRNUV0SDcyenYiXSwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCJ9"

In [512]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': '44085d79-bb1a-48f2-bdd5-df0b190e1227',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['waXGn6xudnrmQba5jMjqNenrQ12v5T1k7XQ5EtH72zv'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [513]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 48KFMNxN2mxGfeDM6SVJKab96xgisTEakEcyeoWjRxbtgDPamdJioGQMJBHess18ZMdfAN6YJ18BSCgTN6PygiDe

Public key: 
 HMgWkf5iUgXtkaGopCehaxQDtawwgAwcDG3vCik7DiEc


**Sending a connection request to acapy**

In [514]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [515]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")
# print("Encoded message: \n")
# pp.pprint(encodedMessage)

In [516]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [517]:
# a bit of a hack to simplify message unpacking,
# decode_pack_message needs a callable object for some reason
def unpackMessage(message, privateKey):
    class FindKey:
        def __init__(self, key):
            self.key = key

        def __call__(self, argument):
            return self.key

    find_key = FindKey(privateKey)   
    return crypto.decode_pack_message(message, find_key)
    
connectionRequestResponseUnpacked = \
    unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "0f0b7e8a-1f5e-4256-953f-97e4bdc3f1f6", "~thread": {"thid": '
 '"827f705f-43fe-457f-bde6-b42c1a3d55be"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"hqZG-0PQq_8prgizR_5rg5kUk_w6q7byc6FtgLG81QOHfUhtgAkRF25kZ9iS_wEcqdhU6h7lENv-_VCdCIwfAQ==", '
 '"sig_data": '
 '"AAAAAF69I-p7IkRJRCI6ICJYRzl0QkVUVUM2dEdjTmQxYWF3VDFHIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpYRzl0QkVUVUM2dEdjTmQxYWF3VDFHIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6WEc5dEJFVFVDNnRHY05kMWFhd1QxRyMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OlhHOXRCRVRVQzZ0R2NOZDFhYXdUMUciLCAicHVibGljS2V5QmFzZTU4IjogIkhWY3JNZXVoY2taV2J0RGphUmtqMlFQSjRhdzJQYmRMdGlVcDdoa3BoeDl3In1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [521]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
#  (this is a hack)replacing artifacts that sometimes happen
sig_data_raw = sig_data_raw.replace("-", "1")
sig_data_raw = sig_data_raw.replace("_", "1")

sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'XG9tBETUC6tGcNd1aawT1G',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:XG9tBETUC6tGcNd1aawT1G#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:XG9tBETUC6tGcNd1aawT1G',
            'publicKey': [{'controller': 'did:sov:XG9tBETUC6tGcNd1aawT1G',
                           'id': 'did:sov:XG9tBETUC6tGcNd1aawT1G#1',
                           'publicKeyBase58': 'HVcrMeuhckZWbtDjaRkj2QPJ4aw2PbdLtiUp7hkphx9w',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:XG9tBETUC6tGcNd1aawT1G;indy',
                         'priority': 0,
                         'recipientKeys': ['HVcrMeuhckZWbtDjaRkj2QPJ4aw2PbdLtiUp7hkphx9w'],
                         'serviceEndpoint': 'http://agent2.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [522]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

def packMessage(message, connection):
    # pass in our private key and recipient key to the encode_pack_message
    decodedRecipientKey = base58.b58decode(connection['DIDDoc']['service'][0]['recipientKeys'][0])
    packedMessage = \
    crypto.encode_pack_message(json.dumps(message), \
                               [decodedRecipientKey], \
                               connection['myKey'])
    return packedMessage.decode('ascii')


pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:XG9tBETUC6tGcNd1aawT1G#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:XG9tBETUC6tGcNd1aawT1G',
             'publicKey': [{'controller': 'did:sov:XG9tBETUC6tGcNd1aawT1G',
                            'id': 'did:sov:XG9tBETUC6tGcNd1aawT1G#1',
                            'publicKeyBase58': 'HVcrMeuhckZWbtDjaRkj2QPJ4aw2PbdLtiUp7hkphx9w',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:XG9tBETUC6tGcNd1aawT1G;indy',
                          'priority': 0,
                          'recipientKeys': ['HVcrMeuhckZWbtDjaRkj2QPJ4aw2PbdLtiUp7hkphx9w'],
                          'serviceEndpoint': 'http://agent2.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Client (admin)',
  'myKey': b'\x9c[\xdc\xce\xcd\xa2b\x8d\x03!\xd6+\x9fR\x19dT\xed<\x

### FEATURE DISCOVERY

In [523]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'f0a4798d-ddf7-4658-a5d7-e65a2e75a777',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-presentation/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/basicmessage/1.0'},
               {'pid': 'https://didcomm.org/present-proof/1.0'},
               {'pid': 'https://didcomm.org/routing/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-static-connections/0.1'},
               {'pid': 'https://didcomm.org/trust_ping/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-issuance/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1'},
               {'pid': 'https://didcomm.org/discover-features

## Toolbox dids

In [ ]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

## Toolbox public did

In [ ]:
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

## Custom plugin / protocol

In [525]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['comment'] = 'qweqwe'

# message['comment'] = "memes"
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '512cd9a1-e3b4-48e6-91e6-99a09e7073ba',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/query',
 'comment': 'qweqwe',
 '~thread': {'thid': 'efd9e368-aaf4-41fe-9237-a480f77f8602'}}
